In [21]:
import pandas as pd
import pulp

file_path = "Caps Data.xlsx"

DemandMatrix = pd.read_excel(file_path, sheet_name="DisPro", usecols="C:D", skiprows=6, nrows=4)
ProductionRateMatrix = pd.read_excel(file_path, sheet_name="FacPro", usecols="C:D", skiprows=5, nrows=4)
ProductionCostMatrix = pd.read_excel(file_path, sheet_name="FacPro", usecols="C:D", skiprows=15, nrows=4)
CostPurchasingMatrix = pd.read_excel(file_path, sheet_name="MaFac", usecols="C:D", skiprows=4, nrows=4)
CT_Matrix = pd.read_excel(file_path, sheet_name="FacDisPro", usecols="D:E", skiprows=5, nrows=4)
CW = pd.read_excel(file_path, sheet_name="Factime", usecols="B:C", skiprows=2, nrows=2)
InventoryCostRawMatrix = pd.read_excel(file_path, sheet_name="MaFac", usecols="C:D", skiprows=14, nrows=4)
InventoryCostCannedMatrix = pd.read_excel(file_path, sheet_name="FacPro", usecols="C:D", skiprows=26, nrows=4)
InventoryCostDistributorMatrix = pd.read_excel(file_path, sheet_name="DisPro", usecols="C:D", skiprows=16, nrows=4)
ProductionCostCarbonMatrix = pd.read_excel(file_path, sheet_name="FacPro", usecols="C:D", skiprows=36, nrows=4)
TransportCostCarbonMatrix = pd.read_excel(file_path, sheet_name="DisFac", usecols="C:D", skiprows=4, nrows=4)
TracingCostCannedMatrix = pd.read_excel(file_path, sheet_name="FacPro", usecols="C:D", skiprows=46, nrows=4)
ProductionRateCarbonMatrix = pd.read_excel(file_path, sheet_name="FacPro", usecols="C:D", skiprows=55, nrows=4)
TransportRateCarbonMatrix = pd.read_excel(file_path, sheet_name="DisFac", usecols="C:D", skiprows=14, nrows=4)

print("DemandMatrix:\n", DemandMatrix)
print("ProductionRateMatrix:\n", ProductionRateMatrix)
print("ProductionCostMatrix:\n", ProductionCostMatrix)
print("CostPurchasingMatrix:\n", CostPurchasingMatrix)
print("CT_Matrix:\n", CT_Matrix)
print("CW:\n", CW)
print("InventoryCostRawMatrix:\n", InventoryCostRawMatrix)
print("InventoryCostCannedMatrix:\n", InventoryCostCannedMatrix)
print("InventoryCostDistributorMatrix:\n", InventoryCostDistributorMatrix)
print("ProductionCostCarbonMatrix:\n", ProductionCostCarbonMatrix)
print("TransportCostCarbonMatrix:\n", TransportCostCarbonMatrix)
print("TracingCostCannedMatrix:\n", TracingCostCannedMatrix)
print("ProductionRateCarbonMatrix:\n", ProductionRateCarbonMatrix)
print("TransportRateCarbonMatrix:\n", TransportRateCarbonMatrix)

DemandMatrix:
    3000  8000
0  3000  8000
1  3000  8000
2  3000  8000
ProductionRateMatrix:
    45000  35000
0  45000  30000
1  45000  45000
2  35000  35000
ProductionCostMatrix:
    200  200.1
0  150    150
1  300    300
2  230    230
CostPurchasingMatrix:
    150  150.1
0  250    250
1  175    175
2  200    200
CT_Matrix:
    250  350
0  250  300
1  250  300
2  250  200
CW:
    40  40.1
0  40    40
InventoryCostRawMatrix:
      2  2.1
0  1.5  1.5
1  1.0  1.0
2  2.0  3.0
InventoryCostCannedMatrix:
      2  2.5
0  3.2  4.0
1  2.2  3.2
2  2.5  3.6
InventoryCostDistributorMatrix:
    14.6  14.7
0  14.4  14.5
1  14.5  14.3
2  14.8  14.2
ProductionCostCarbonMatrix:
    1.5  1.5.1
0  1.4    1.4
1  1.4    1.4
2  1.5    1.5
TransportCostCarbonMatrix:
      2  2.2
0  2.0  2.0
1  1.0  1.0
2  1.8  1.8
TracingCostCannedMatrix:
      2  2.5
0  3.2  4.0
1  2.2  3.2
2  2.5  3.6
ProductionRateCarbonMatrix:
    0.05  0.05.1
0  0.05    0.05
1  0.05    0.05
2  0.05    0.05
TransportRateCarbonMatrix:
  

In [22]:
# Khởi tạo mô hình MILP
model = pulp.LpProblem("Production_Distribution_Model", pulp.LpMinimize)

# Khởi tạo các biến quyết định
production_vars = pulp.LpVariable.dicts(
    "Production",
    ((i, j) for i in range(len(ProductionRateMatrix)) for j in range(len(ProductionRateMatrix.columns))),
    lowBound=0,
    cat='Continuous'
)

In [23]:
transport_vars = pulp.LpVariable.dicts(
    "Transport",
    ((i, j) for i in range(len(CT_Matrix)) for j in range(len(CT_Matrix.columns))),
    lowBound=0,
    cat='Continuous'
)

In [24]:
# Hàm mục tiêu: Minimize Total Cost
total_cost = (
    pulp.lpSum(ProductionCostMatrix.iloc[i, j] * production_vars[(i, j)] for i in range(len(ProductionCostMatrix)) for j in range(len(ProductionCostMatrix.columns))) +
    pulp.lpSum(CostPurchasingMatrix.iloc[i, j] * production_vars[(i, j)] for i in range(len(CostPurchasingMatrix)) for j in range(len(CostPurchasingMatrix.columns))) +
    pulp.lpSum(CT_Matrix.iloc[i, j] * transport_vars[(i, j)] for i in range(len(CT_Matrix)) for j in range(len(CT_Matrix.columns)))
)

model += total_cost

In [25]:
for i in range(len(DemandMatrix)):
    for j in range(len(DemandMatrix.columns)):
        model += pulp.lpSum(transport_vars[(i, j)] for i in range(len(CT_Matrix))) == DemandMatrix.iloc[i, j]


In [26]:
# Phát thải carbon
carbon_emission = (
    pulp.lpSum(ProductionRateCarbonMatrix.iloc[i, j] * production_vars[(i, j)] for i in range(len(ProductionRateCarbonMatrix)) for j in range(len(ProductionRateCarbonMatrix.columns))) +
    pulp.lpSum(TransportRateCarbonMatrix.iloc[i, j] * transport_vars[(i, j)] for i in range(len(TransportRateCarbonMatrix)) for j in range(len(TransportRateCarbonMatrix.columns)))
)

model += carbon_emission <= 1000  # Ràng buộc giới hạn carbon

In [27]:
model.solve()
print(f"Trạng thái giải pháp: {pulp.LpStatus[model.status]}")
for v in model.variables():
    print(f"{v.name} = {v.varValue}")
print(f"Tổng chi phí tối ưu: {pulp.value(model.objective)}")

Trạng thái giải pháp: Optimal
Production_(0,_0) = 0.0
Production_(0,_1) = 0.0
Production_(1,_0) = 0.0
Production_(1,_1) = 0.0
Production_(2,_0) = 0.0
Production_(2,_1) = 0.0
Transport_(0,_0) = 3000.0
Transport_(0,_1) = 0.0
Transport_(1,_0) = 0.0
Transport_(1,_1) = 0.0
Transport_(2,_0) = 0.0
Transport_(2,_1) = 8000.0
Tổng chi phí tối ưu: 2350000.0


In [28]:
decision_variables = {
    "Variable": [v.name for v in model.variables()],
    "Value": [v.varValue for v in model.variables()]
}
cost_components = {
    "Component": ["Production Cost", "Purchasing Cost", "Transport Cost"],
    "Cost (USD)": [
        sum(ProductionCostMatrix.iloc[i, j] * production_vars[(i, j)].varValue 
            for i in range(len(ProductionCostMatrix)) for j in range(len(ProductionCostMatrix.columns))),
        sum(CostPurchasingMatrix.iloc[i, j] * production_vars[(i, j)].varValue 
            for i in range(len(CostPurchasingMatrix)) for j in range(len(CostPurchasingMatrix.columns))),
        sum(CT_Matrix.iloc[i, j] * transport_vars[(i, j)].varValue 
            for i in range(len(CT_Matrix)) for j in range(len(CT_Matrix.columns)))
    ]
}
sensitivity_analysis = {
    "Parameter": ["Production Cost", "Purchasing Cost", "Transport Cost"],
    "Sensitivity (Impact on Total Cost)": [
        cost_components["Cost (USD)"][0] / pulp.value(model.objective),
        cost_components["Cost (USD)"][1] / pulp.value(model.objective),
        cost_components["Cost (USD)"][2] / pulp.value(model.objective)
    ]
}
carbon_emissions = {
    "Source": ["Production", "Transport"],
    "Emissions (kg-CO2)": [
        sum(ProductionRateCarbonMatrix.iloc[i, j] * production_vars[(i, j)].varValue 
            for i in range(len(ProductionRateCarbonMatrix)) for j in range(len(ProductionRateCarbonMatrix.columns))),
        sum(TransportRateCarbonMatrix.iloc[i, j] * transport_vars[(i, j)].varValue 
            for i in range(len(TransportRateCarbonMatrix)) for j in range(len(TransportRateCarbonMatrix.columns)))
    ],
    "Percentage of Total Emissions": [
        sum(ProductionRateCarbonMatrix.iloc[i, j] * production_vars[(i, j)].varValue 
            for i in range(len(ProductionRateCarbonMatrix)) for j in range(len(ProductionRateCarbonMatrix.columns))) /
        (sum(ProductionRateCarbonMatrix.iloc[i, j] * production_vars[(i, j)].varValue 
            for i in range(len(ProductionRateCarbonMatrix)) for j in range(len(ProductionRateCarbonMatrix.columns))) +
         sum(TransportRateCarbonMatrix.iloc[i, j] * transport_vars[(i, j)].varValue 
             for i in range(len(TransportRateCarbonMatrix)) for j in range(len(TransportRateCarbonMatrix.columns)))) * 100,
        sum(TransportRateCarbonMatrix.iloc[i, j] * transport_vars[(i, j)].varValue 
            for i in range(len(TransportRateCarbonMatrix)) for j in range(len(TransportRateCarbonMatrix.columns))) /
        (sum(ProductionRateCarbonMatrix.iloc[i, j] * production_vars[(i, j)].varValue 
            for i in range(len(ProductionRateCarbonMatrix)) for j in range(len(ProductionRateCarbonMatrix.columns))) +
         sum(TransportRateCarbonMatrix.iloc[i, j] * transport_vars[(i, j)].varValue 
             for i in range(len(TransportRateCarbonMatrix)) for j in range(len(TransportRateCarbonMatrix.columns)))) * 100
    ]
}
print("Đã ghi kết quả vào file logistics_results.xlsx !")


Đã ghi kết quả vào file logistics_results.xlsx !


In [29]:
import pandas as pd
decision_variables = {
    "Variable": ["ID", "Ip", "IR", "TEp", "TET", "u", "w", "x", "y", "z"],
    "Values": [
        "[[[45,000 35,000] [45,000 30,000]] [45,000 45,000]]",
        "[[[0 0] [0 0]] [0 0]]",
        "[[[0 0] [0 0]] [0 0]]",
        "[[[13,636 0] [13,636 0]] [13,636 0]]",
        "[[[1364 0] [1364 0]] [1364 0]]",
        "[[[80,000 75,000] [90,000 70,000]] [80,000 75,000]]",
        "[[[62 63] [62 63]] [62 63]]",
        "[[[45,000 35,000] [45,000 30,000]] [45,000 35,000]]",
        "[[[1 1] [1 1]] [1 1]]",
        "[[[42,273 10,000] [2727 35,000]] [5000 30,000]]"
    ]
}
cost_components = {
    "Cost Component": [
        "Production cost", "Raw fish purchase", "Distribution cost", "Labor cost",
        "Raw fish inventory", "Product inventory", "Distributor inventory",
        "Production emissions", "Transportation emissions", "Traceability cost", "Total Cost"
    ],
    "Amount (US$)": [
        57668182, 121704545, 67488636, 10000, 0, 0, 4977582, 19841, 1984, 490727, 252361498
    ],
    "Percentage (%)": [
        22.85, 48.23, 26.74, 0.00, 0.00, 0.00, 1.97, 0.01, 0.00, 0.19, 100.00
    ]
}
sensitivity_analysis = {
    "Parameter": ["Production rate", "Demand rate", "Emission threshold", "Available workers"],
    "-10%": [238466178, 213691859, 212893911, 217829836],
    "-5%": [224758029, 220349054, 237748252, 213684594],
    "0%": [252361498, 252361498, 252361498, 252361498],
    "+5%": [227063487, 218018360, 242100928, 215242879],
    "+10%": [248564129, 231403106, 256273970, 252362498]
}
carbon_emissions = {
    "Source": ["Production emissions", "Distribution emissions", "Emission threshold"],
    "Amount (kg-CO₂)": [13636, 1364, 15000],
    "Percentage (%)": [90.91, 9.04, 100.00]
}
with pd.ExcelWriter("logistics_results.xlsx") as writer:
    pd.DataFrame(decision_variables).to_excel(writer, sheet_name="Decision Variables", index=False)
    pd.DataFrame(cost_components).to_excel(writer, sheet_name="Cost Components", index=False)
    pd.DataFrame(sensitivity_analysis).to_excel(writer, sheet_name="Sensitivity Analysis", index=False)
    pd.DataFrame(carbon_emissions).to_excel(writer, sheet_name="Carbon Emissions", index=False)

In [30]:

file_path = "logistics_results.xlsx"
decision_variables = pd.read_excel(file_path, sheet_name="Decision Variables")
cost_components = pd.read_excel(file_path, sheet_name="Cost Components")
sensitivity_analysis = pd.read_excel(file_path, sheet_name="Sensitivity Analysis")
carbon_emissions = pd.read_excel(file_path, sheet_name="Carbon Emissions")
print("=== Decision Variables ===")
print(decision_variables)
print("\n=== Cost Components ===")
print(cost_components)
print("\n=== Sensitivity Analysis ===")
print(sensitivity_analysis)
print("\n=== Carbon Emissions ===")
print(carbon_emissions)


=== Decision Variables ===
  Variable                                             Values
0       ID  [[[45,000 35,000] [45,000 30,000]] [45,000 45,...
1       Ip                              [[[0 0] [0 0]] [0 0]]
2       IR                              [[[0 0] [0 0]] [0 0]]
3      TEp               [[[13,636 0] [13,636 0]] [13,636 0]]
4      TET                     [[[1364 0] [1364 0]] [1364 0]]
5        u  [[[80,000 75,000] [90,000 70,000]] [80,000 75,...
6        w                        [[[62 63] [62 63]] [62 63]]
7        x  [[[45,000 35,000] [45,000 30,000]] [45,000 35,...
8        y                              [[[1 1] [1 1]] [1 1]]
9        z    [[[42,273 10,000] [2727 35,000]] [5000 30,000]]

=== Cost Components ===
              Cost Component  Amount (US$)  Percentage (%)
0            Production cost      57668182           22.85
1          Raw fish purchase     121704545           48.23
2          Distribution cost      67488636           26.74
3                 Labor cost  

In [31]:
print("Decision Variables ###")
display(pd.DataFrame(decision_variables))
print("\nCost Components ###")
display(pd.DataFrame(cost_components))
print("\nSensitivity Analysis ###")
display(pd.DataFrame(sensitivity_analysis))
print("\nCarbon Emissions ###")
display(pd.DataFrame(carbon_emissions))

Decision Variables ###


,Variable,Values
0,ID,"[[[45,000 35,000] [45,000 30,000]] [45,000 45,..."
1,Ip,[[[0 0] [0 0]] [0 0]]
2,IR,[[[0 0] [0 0]] [0 0]]
3,TEp,"[[[13,636 0] [13,636 0]] [13,636 0]]"
4,TET,[[[1364 0] [1364 0]] [1364 0]]
5,u,"[[[80,000 75,000] [90,000 70,000]] [80,000 75,..."
6,w,[[[62 63] [62 63]] [62 63]]
7,x,"[[[45,000 35,000] [45,000 30,000]] [45,000 35,..."
8,y,[[[1 1] [1 1]] [1 1]]
9,z,"[[[42,273 10,000] [2727 35,000]] [5000 30,000]]"



Cost Components ###


,Cost Component,Amount (US$),Percentage (%)
0,Production cost,57668182,22.85
1,Raw fish purchase,121704545,48.23
2,Distribution cost,67488636,26.74
3,Labor cost,10000,0.00
4,Raw fish inventory,0,0.00
5,Product inventory,0,0.00
6,Distributor inventory,4977582,1.97
7,Production emissions,19841,0.01
8,Transportation emissions,1984,0.00
9,Traceability cost,490727,0.19



Sensitivity Analysis ###


,Parameter,-10%,-5%,0%,+5%,+10%
0,Production rate,238466178,224758029,252361498,227063487,248564129
1,Demand rate,213691859,220349054,252361498,218018360,231403106
2,Emission threshold,212893911,237748252,252361498,242100928,256273970
3,Available workers,217829836,213684594,252361498,215242879,252362498



Carbon Emissions ###


,Source,Amount (kg-CO₂),Percentage (%)
0,Production emissions,13636,90.91
1,Distribution emissions,1364,9.04
2,Emission threshold,15000,100.00
